# 武将カフェ検索＆予約プラグインデモ(Python)

## Setup
以下のプラグインを起動する。
- cafe-review-plugin
- restaurant-reservation-plugin

それぞれのプラグインディレクトリに入って、以下を実行します。

```bash
pip install -r requirements.txt
```

ライブラリのインストールができたら、

```bash
python main.py
```

でサーバーを起動します。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools import AIPluginTool
from langchain.agents.mrkl import prompt

In [ ]:
#If Azure OpenAI
#import os
#os.environ["OPENAI_API_TYPE"] = "azure"
#os.environ["OPENAI_API_KEY"] = "<Your Azure OpenAI KEY>"
#os.environ["OPENAI_API_BASE"] = "https://<Your Azure OpenAI Serivce>.openai.azure.com/"

In [ ]:
OPENAI_API_KEY = "<Your OpenAI Key>"

## Tool のロード
ChatGPT Plugin をロードするには、AIPluginTool に登録する

In [ ]:
#llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0, model_kwargs={"deployment_id":"gpt-4-0613"})　# Azure OpenAI
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)
tools = load_tools(["requests_all"])
plugin_urls = ["http://localhost:5005/.well-known/ai-plugin.json", "http://localhost:5006/.well-known/ai-plugin.json"]

tools += [AIPluginTool.from_plugin_url(url) for url in plugin_urls]

## Agent の初期化

Agent には `ZERO_SHOT_REACT_DESCRIPTION` を利用する。制約事項については SUFFIX を追加している。


In [ ]:
SUFFIX = """
'Answer should be in Japanese. Use http instead of https for endpoint.
If there is no year in the reservation, use the year 2023. 
"""

# Responsible AI MetaPrompt
#**IMPORTANT**
#If a restaurant reservation is available, must check with the user before making a reservation if yes.'

agent_chain = initialize_agent(tools,
                            llm,
                            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
                            agent_kwargs=dict(suffix=SUFFIX + prompt.SUFFIX))


In [ ]:
agent_chain.run("源範頼に関連するカフェ名を検索して")

In [ ]:
agent_chain.run("源範頼に関連するカフェ名を検索して、7/1の18時に予約に空きがあるか教えて。もし空いていたら予約しておいて。")

In [ ]:
agent_chain.run("カフェかば殿　に7/1の18時に予約を取って。")